# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Import stuff

In [31]:
import graphlab
import pandas as pd
import numpy as np
from sklearn import linear_model  # using scikit-learn
from math import log, sqrt


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [82]:
gl_sales = graphlab.SFrame('kc_house_data.gl/')

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [83]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [208]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [212]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

#model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
#                                             validation_set=None, 
#                                             l2_penalty=0., l1_penalty=1e10)



Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [213]:
print model_all.sparse_coef_
print all_features[3]
print all_features[10]
print all_features[12]

coeficientes = pd.DataFrame(model_all.coef_,index=all_features,columns=['values'])
print coeficientes


  (0, 3)	134.43931395541435
  (0, 10)	24750.00458560952
  (0, 12)	61749.10309070811
sqft_living
view
grade
                        values
bedrooms              0.000000
bedrooms_square       0.000000
bathrooms             0.000000
sqft_living         134.439314
sqft_living_sqrt      0.000000
sqft_lot              0.000000
sqft_lot_sqrt         0.000000
floors                0.000000
floors_square         0.000000
waterfront            0.000000
view              24750.004586
condition             0.000000
grade             61749.103091
sqft_above            0.000000
sqft_basement         0.000000
yr_built             -0.000000
yr_renovated          0.000000


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [57]:
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)
testing= pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)

(training_and_validation, gl_testing) = gl_sales.random_split(.9,seed=1) # initial train/test split
(gl_training, gl_validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [58]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

gl_testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
gl_testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
gl_testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
gl_testing['floors_square'] = testing['floors']*testing['floors']

gl_training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
gl_training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
gl_training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
gl_training['floors_square'] = training['floors']*training['floors']

gl_validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
gl_validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
gl_validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
gl_validation['floors_square'] = validation['floors']*validation['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [99]:
error=np.empty(len(np.logspace(1,7,num=12)))
i=0
for l1_penalty in np.logspace(1,7,num=12):
    model_all = linear_model.Lasso(alpha=l1_penalty, normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    
    error[i]=((model_all.predict(validation[all_features])-validation['price'])**2).sum()
    i=i+1


In [100]:
print error


[3.98213327e+14 4.00975454e+14 4.34478867e+14 4.92880611e+14
 7.71413707e+14 1.22250686e+15 1.22250686e+15 1.22250686e+15
 1.22250686e+15 1.22250686e+15 1.22250686e+15 1.22250686e+15]


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [104]:
train2_model = linear_model.Lasso(alpha=10.0, normalize=True) # set parameters
train2_model.fit(training[all_features], training['price']) # learn weights

Lasso(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [107]:
train_model_2.intercept_
print 'Cantidad de pesos no nulos es', np.count_nonzero(train2_model.coef_)+1 #<-- 1 por el intercept


Cantidad de pesos no nulos es 15


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [110]:
l1_penalty_values = np.logspace(8, 10, num=20)
print l1_penalty_values

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [140]:
l1_penalty_values = np.logspace(1, 4, num=20)


In [141]:
counts_of_nonzeros = []
for l1_penalty in l1_penalty_values:
    model_all = linear_model.Lasso(alpha=l1_penalty, normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    
    if model_all.intercept_==0:
        counts = np.count_nonzero(model_all.coef_)
    else:
        counts = np.count_nonzero(model_all.coef_)+1
        
    counts_of_nonzeros.append((l1_penalty,counts))
    
    

In [143]:
print counts_of_nonzeros

[(10.0, 15), (14.38449888287663, 15), (20.6913808111479, 15), (29.76351441631318, 15), (42.81332398719393, 13), (61.58482110660264, 12), (88.58667904100822, 11), (127.42749857031335, 10), (183.29807108324357, 7), (263.6650898730358, 6), (379.26901907322497, 6), (545.5594781168514, 6), (784.7599703514607, 5), (1128.8378916846884, 3), (1623.776739188721, 3), (2335.7214690901214, 2), (3359.818286283781, 1), (4832.930238571752, 1), (6951.927961775606, 1), (10000.0, 1)]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [160]:
max_nonzeros = 7
l1_min_list=[n for n,m in counts_of_nonzeros if m>max_nonzeros]
l1_max_list=[n for n,m in counts_of_nonzeros if m<max_nonzeros]

In [214]:
l1_penalty_min = max(l1_min_list) 
l1_penalty_max = min(l1_max_list)
print round(l1_penalty_min),round(l1_penalty_max)

127.0 264.0


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [199]:
info_list=[]
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model_all = linear_model.Lasso(alpha=l1_penalty, normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    
    sparsity = np.count_nonzero(model_all.coef_)
    if (model_all.intercept_ != 0):
        sparsity +=1
    
    if sparsity  == max_nonzeros:
        error = (model_all.predict(validation[all_features])-validation['price'])**2
        
        rss=error.sum()
        
        info_list.append((l1_penalty,rss))

rss_list=[m for n,m in info_list] 
print info_list[rss_list.index(min(rss_list))]

(156.10909673930755, 440037365263316.56)


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [207]:
model_all = linear_model.Lasso(alpha=info_list[0][0], normalize=True) # set parameters
model_all.fit(training[all_features], training['price']) # learn weights
    
coeficientes = pd.DataFrame(model_all.coef_,index=all_features,columns=['values'])
print coeficientes[coeficientes['values']!=0.]


                    values
bathrooms     10610.890284
sqft_living     163.380252
waterfront   506451.687115
view          41960.043555
grade        116253.553700
yr_built      -2612.234880
